In [12]:
import psycopg2
import pandas as pd
import os

In [2]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "database-2.cvu49ggyv9ee.eu-north-1.rds.amazonaws.com",
    "database"  : "postgres",
    "user"      : "postgres",
    "password"  : "postgres"
}

In [3]:
#Function to establish connect to potsgres database
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [4]:
#Function to query a postgres database - Params: Conn, query and column names
def postgresql_to_dataframe(conn, select_query, column_names):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    return df

In [13]:
def copy_from_file(conn, df, table):
    """
    Here we are going save the dataframe on disk as 
    a csv file, load the csv file  
    and use copy_from() to copy it to the table
    """
    # Save the dataframe to disk
    tmp_df = "./tmp_dataframe.csv"
    df.to_csv(tmp_df, index_label='id', header=False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    os.remove(tmp_df)

In [ ]:
#conn = connect(param_dic) # connect to the database
#copy_from_file(conn,df , 'table_name') # copy the dataframe to SQL
#conn.close()

In [5]:
# Connect to the database
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


In [6]:
column_names = ['Date Received', 'Forecast_Region', 'Time', 'Product', 'Type', 'Actuals']

In [9]:
#Define the query
query = "select * from forecast"

In [11]:

# Execute the "SELECT *" query
df = postgresql_to_dataframe(conn, query, column_names)
df.head()

,Date Received,Forecast_Region,Time,Product,Type,Actuals
0,2020-01-02,UK,Q1 2020,G360 CDx,Commercial,1
1,2020-01-02,UK,Q1 2020,G360 CDx,Commercial,1
2,2020-01-02,Turkey,Q1 2020,G360 CDx,Commercial,1
3,2020-01-02,Turkey,Q1 2020,G360 CDx,Commercial,1
4,2020-01-02,Spain,Q1 2020,G360 CDx,Commercial,1
